In [5]:
import cirq
import numpy as np
try:
    from cirqprojectq.circ_engine import CIRQ
except:
    import os
    os.chdir("../../")
    from cirqprojectq.circ_engine import CIRQ
import projectq
qubits = [cirq.google.XmonQubit(0, i) for i in range(2)]
mycirq = CIRQ(qubits=qubits)
eng = projectq.MainEngine(backend=mycirq)
qureg = eng.allocate_qureg(len(qubits))
eng.flush()
projectq.ops.Rx(0.25 * np.pi) | qureg[0]
projectq.ops.Ry(0.5 * np.pi) | qureg[1]
projectq.ops.Rz(0.5 * np.pi) | qureg[0]
projectq.ops.H | qureg[1]
projectq.ops.C(projectq.ops.X) | (qureg[0], qureg[1])
projectq.ops.Z | qureg[0]
projectq.ops.X | qureg[0]
projectq.ops.Y | qureg[0]
eng.flush()
print(mycirq.circuit, "\n\n")
for m in mycirq.circuit.moments:
    print(m)

(0, 0): ───X^0.25───Z^0.5───────@───Z───X───Y───
                                │
(0, 1): ────────────Y^0.5───H───X─────────────── 


X**0.24999999999985723((0, 0))
Z**0.5000000000000329((0, 0)) and Y**0.5000000000000329((0, 1))
H((0, 1))
CX((0, 0), (0, 1))
Z((0, 0))
X((0, 0))
Y((0, 0))


In [2]:
import cirqprojectq
from cirqprojectq.xmon_decompositions import all_defined_decomposition_rules as xmondec

def is_supported(eng, cmd):
    if isinstance(cmd.gate, projectq.ops.ClassicalInstructionGate):
        # This is required to allow Measure, Allocate, Deallocate, Flush
        return True
    elif isinstance(cmd.gate, cirqprojectq.xmon_gates.XmonGate):
        return True
    else:
        return False

supported_gate_set_filter = projectq.cengines.InstructionFilter(is_supported)
ruleset = projectq.cengines.DecompositionRuleSet(xmondec)
replacer = projectq.cengines.AutoReplacer(ruleset)
engine_list = [replacer, supported_gate_set_filter]

print("Circuit in Xmon native gates:")
eng = projectq.MainEngine(backend=projectq.backends.Simulator(), engine_list=engine_list+[projectq.backends.CommandPrinter()])
qureg = eng.allocate_qureg(2)
projectq.ops.H | qureg[0]
projectq.ops.H | qureg[1]
projectq.ops.C(projectq.ops.X) | (qureg[0], qureg[1])
projectq.ops.All(projectq.ops.Measure) | qureg
eng.flush()
print("Measurement outcome:", [int(q) for q in qureg])

Circuit in Xmon native gates:
Allocate | Qureg[0]
Allocate | Qureg[1]
W(0.5, 0.5) | Qureg[0]
ExpZ(1.0) | Qureg[0]
W(0.5, 0.5) | Qureg[1]
ExpZ(1.0) | Qureg[1]
W(0.5, 0.5) | Qureg[1]
ExpZ(1.0) | Qureg[1]
Exp11(1.0) | ( Qureg[0], Qureg[1] )
W(0.5, 0.5) | Qureg[1]
ExpZ(1.0) | Qureg[1]
Measure | Qureg[0]
Measure | Qureg[1]
Measurement outcome: [1, 0]


In [6]:
qubits = [cirq.google.XmonQubit(0, i) for i in range(2)]
mycirq = CIRQ(qubits=qubits)
eng = projectq.MainEngine(backend=mycirq, engine_list=engine_list)
qureg = eng.allocate_qureg(len(qubits))
eng.flush()
projectq.ops.Rx(0.25 * np.pi) | qureg[0]
projectq.ops.Ry(0.5 * np.pi) | qureg[1]
projectq.ops.Rz(0.5 * np.pi) | qureg[0]
projectq.ops.H | qureg[1]
projectq.ops.C(projectq.ops.X) | (qureg[0], qureg[1])
projectq.ops.Z | qureg[0]
projectq.ops.X | qureg[0]
projectq.ops.Y | qureg[0]
eng.flush()
print(mycirq.circuit, "\n\n")
for m in mycirq.circuit.moments:
    print(m)

(0, 0): ───X^0.25───Z^0.5─────────────────────────────@───────────Z^-1.0───X───Y───
                                                      │
(0, 1): ───Y^0.5────Y^0.5───Z^-1.0───Y^0.5───Z^-1.0───Z───Y^0.5───Z^-1.0─────────── 


X^0.2499999999998573((0, 0)) and Y^0.500000000000033((0, 1))
Z^0.500000000000033((0, 0)) and Y^0.5((0, 1))
Z^-0.999999999999934((0, 1))
Y^0.5((0, 1))
Z^-0.999999999999934((0, 1))
CZ((0, 0), (0, 1))
Y^0.5((0, 1))
Z^-0.999999999999934((0, 1)) and Z^-0.999999999999934((0, 0))
X((0, 0))
Y((0, 0))
